
# Multi-Criteria Analysis (MCA) Lab — Net‑Zero 2050 Policy Options

**Context:** You are a consultant hired by the government of your selected country. Your task is to provide evidence-based policy advice on pathways to achieve net‑zero (or near‑zero) emissions by **2050**.

This lab helps you build a transparent **Multi‑Criteria Analysis (MCA)** to compare policy alternatives using **technical, economic, environmental, social, and institutional** criteria.

---

## Learning goals

By completing this lab, you will be able to:

- Define a decision problem and select appropriate **criteria**.
- **Normalize** heterogeneous indicators to a common 0–100 scale.
- **Weight** criteria (direct weights or data‑driven/entropy weights).
- Compute an **aggregate index** (Weighted Sum) and optional **TOPSIS** score.
- Interpret **results and trade‑offs**, and reflect on **limitations** of modeling.
- Link quantitative model outputs to **social** and **environmental** factors.
- Cross‑check conclusions against **SDGs** (to 2050) and relevant evidence (e.g., NDCs, EU Green Deal/Fit for 55, sectoral studies, public surveys).

---

## Project frame (what you will use this notebook for)

Start from key policy documents (NDCs, long‑term strategies), EU‑level commitments (Green Deal, Fit for 55, Climate Law), and quantitative insights from ECEMF energy models (e.g., **net‑zero**, **NPI**). Use MCA to evaluate **environmental**, **social**, and **economic** impacts, and integrate complementary evidence (e.g., **SDGs**, public acceptance, sectoral studies).

**Important:** While SDGs focus on 2030, build indicators that track to **2050**.

---

### Workflow in this notebook

1. Decision & evaluation criteria (define alternatives and indicators)
2. Normalization (0–100)
3. Weighting (direct or entropy; optional pairwise template provided)
4. Aggregate index (Weighted Sum; optional TOPSIS)
5. Results, sensitivity checks, and conclusions

> 🧠 *Pedagogical tip:* Keep your model **auditable**. The notebook is structured so you can plug in your own alternatives, criteria, and data without editing core logic.



## 0) Setup

Run the cell below to (re)install required packages (safe in Colab).

In [1]:

# If you're running in Colab, this is safe to execute.
# If packages are already installed, pip will skip or update as needed.
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
except Exception:
    pass

!pip -q install pandas numpy matplotlib openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_colwidth", 120)



## 1) Decision & evaluation criteria

In MCA we compare **alternatives** (policy packages, pathways, instruments) against **criteria** (indicators).

- **Alternatives** — examples: *NZ2050 (deep electrification), NZ2050 (balanced fuels), NPI (policies in place)*.
- **Criteria** — examples across five pillars:
  - **Technical:** grid reliability index, renewable share, storage capacity added.
  - **Economic:** total system cost (%GDP), average household energy bill (€), CAPEX needs.
  - **Environmental:** lifecycle GHG (Mt), air pollutant index, biodiversity pressure score.
  - **Social:** employment change in coal regions, public acceptance index, energy poverty rate.
  - **Institutional:** implementation readiness, regulatory complexity, investment risk score.

> 🎯 Your job: choose criteria that are **measurable**, **policy‑relevant**, and **non‑redundant**. If in doubt, include a brief justification.

### Input data options

- **Option A (quick start):** Edit the example dataframe directly in Python.
- **Option B (CSV/Excel):** Download a template → fill in your values → upload → load here.

Each criterion must specify a **direction**:
- `"benefit"`: higher values are *better* (e.g., renewable share %).
- `"cost"`: higher values are *worse* (e.g., total system cost %GDP).

We will normalize all criteria to a **0–100** scale (100 = best).

In [2]:

import pandas as pd

# ---------- Quick-start example (EDIT ME) ----------
# Rows = alternatives (policy options); Columns = criteria (raw values)
raw_data = pd.DataFrame({
    "Alternative": ["NZ2050_DeepElec", "NZ2050_Balanced", "NPI_Current"],
    # TECHNICAL
    "Renewable_share_%": [85, 75, 55],
    "Annual_storage_additions_GWh": [25, 18, 6],
    # ECONOMIC
    "System_cost_pct_GDP": [4.2, 3.9, 3.2],
    "Household_bill_EUR": [1200, 1300, 1500],
    # ENVIRONMENTAL
    "GHG_2050_MtCO2e": [30, 60, 220],
    "Air_pollution_index": [20, 30, 60],
    # SOCIAL
    "Jobs_change_coal_regions_thousand": [+45, +30, -10],
    "Energy_poverty_rate_%": [6, 7, 10],
    # INSTITUTIONAL
    "Implementation_readiness_score": [70, 65, 55],
    "Regulatory_complexity_index": [30, 35, 50]
})

# For each criterion column above, specify its direction: "benefit" or "cost"
criterion_meta = pd.DataFrame({
    "criterion": [
        "Renewable_share_%",
        "Annual_storage_additions_GWh",
        "System_cost_pct_GDP",
        "Household_bill_EUR",
        "GHG_2050_MtCO2e",
        "Air_pollution_index",
        "Jobs_change_coal_regions_thousand",
        "Energy_poverty_rate_%",
        "Implementation_readiness_score",
        "Regulatory_complexity_index",
    ],
    "direction": [
        "benefit",   # Renewable_share_%
        "benefit",   # Annual_storage_additions_GWh
        "cost",      # System_cost_pct_GDP
        "cost",      # Household_bill_EUR
        "cost",      # GHG_2050_MtCO2e
        "cost",      # Air_pollution_index
        "benefit",   # Jobs_change_coal_regions_thousand
        "cost",      # Energy_poverty_rate_%
        "benefit",   # Implementation_readiness_score
        "cost",      # Regulatory_complexity_index
    ]
})

print("▶️ Edit 'raw_data' and 'criterion_meta' above for your case, or use the template below.")
raw_data


▶️ Edit 'raw_data' and 'criterion_meta' above for your case, or use the template below.


,Alternative,Renewable_share_%,Annual_storage_additions_GWh,System_cost_pct_GDP,Household_bill_EUR,GHG_2050_MtCO2e,Air_pollution_index,Jobs_change_coal_regions_thousand,Energy_poverty_rate_%,Implementation_readiness_score,Regulatory_complexity_index
0,NZ2050_DeepElec,85,25,4.2,1200,30,20,45,6,70,30
1,NZ2050_Balanced,75,18,3.9,1300,60,30,30,7,65,35
2,NPI_Current,55,6,3.2,1500,220,60,-10,10,55,50



### Downloadable template (optional)

Run the cell to save a CSV template to your working directory. Fill it and upload in Colab, then load using the next cell.


In [3]:

template = raw_data.copy()
template.to_csv("mca_raw_template.csv", index=False)
criterion_meta.to_csv("mca_criterion_meta_template.csv", index=False)
from google.colab import files  # safe to import in Colab; in local Jupyter, ignore if not available

print("Saved: mca_raw_template.csv and mca_criterion_meta_template.csv")
try:
    files.download("mca_raw_template.csv")
    files.download("mca_criterion_meta_template.csv")
except Exception as e:
    print("If not in Colab, manual download is not available here. Files are saved in the working directory.")


ModuleNotFoundError: No module named 'google'


### Load your own data (if you used the template)

Upload your completed CSVs in the Colab sidebar (Files) and set the paths below.


In [ ]:

# Set to your uploaded file paths if using templates
USE_UPLOADED = False
RAW_PATH = "mca_raw_template.csv"
META_PATH = "mca_criterion_meta_template.csv"

if USE_UPLOADED:
    raw_data = pd.read_csv(RAW_PATH)
    criterion_meta = pd.read_csv(META_PATH)

display(raw_data.head())
display(criterion_meta)



## 2) Normalization (0–100 scale)

Why normalize? Criteria have different units and ranges (€, %, MtCO₂e, indices). To compare them, we transform raw values to a common **0–100** scale where **100 = best** given the criterion's **direction**.

We use **min–max normalization** by default:
- For **benefit** criteria:  
  `score = 100 * (x - min) / (max - min)`
- For **cost** criteria:  
  `score = 100 * (max - x) / (max - min)`

> ⚠️ *Check your min & max*: If your options are too similar, consider adding external targets (e.g., EU benchmarks or SDGs) as anchor values to avoid inflated scores.


In [ ]:

import numpy as np

def normalize_minmax(df_raw: pd.DataFrame, meta: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()
    df_norm = pd.DataFrame({"Alternative": df["Alternative"]})
    for _, row in meta.iterrows():
        c = row["criterion"]
        direction = row["direction"].strip().lower()
        values = df[c].astype(float)
        cmin, cmax = values.min(), values.max()
        if np.isclose(cmax, cmin):
            # Avoid divide-by-zero: assign 100 for benefit or 0 for cost (neutral handling)
            if direction == "benefit":
                df_norm[c] = 100.0
            elif direction == "cost":
                df_norm[c] = 0.0
            else:
                raise ValueError(f"Unknown direction for {c}: {direction}")
            continue
        if direction == "benefit":
            df_norm[c] = 100.0 * (values - cmin) / (cmax - cmin)
        elif direction == "cost":
            df_norm[c] = 100.0 * (cmax - values) / (cmax - cmin)
        else:
            raise ValueError(f"Unknown direction for {c}: {direction}")
    return df_norm

normalized = normalize_minmax(raw_data, criterion_meta)
print("Normalized scores (0–100, 100 = best):")
normalized



## 3) Weighting the criteria

Because not all criteria are equally important, we apply **weights** that sum to 1. Two options:

- **Direct weights (recommended for transparency):** You assign weights per criterion (e.g., 0.20 to environmental GHG, 0.15 to social acceptance, ...).  
- **Entropy weights (optional):** A data-driven method that gives higher weight to criteria with more discriminative power across alternatives.

> ✍️ *Pedagogical note:* Document your **rationale** for weights (e.g., policy priorities, stakeholder workshops, public surveys, SDG alignment). Consider testing multiple weight sets to reveal trade-offs.


In [ ]:

# --- Option A: Direct weights (EDIT ME to reflect your priorities) ---
# Provide weights for every criterion in 'criterion_meta'. They must sum to 1.
direct_weights = {
    "Renewable_share_%": 0.10,
    "Annual_storage_additions_GWh": 0.08,
    "System_cost_pct_GDP": 0.15,
    "Household_bill_EUR": 0.10,
    "GHG_2050_MtCO2e": 0.25,
    "Air_pollution_index": 0.10,
    "Jobs_change_coal_regions_thousand": 0.10,
    "Energy_poverty_rate_%": 0.06,
    "Implementation_readiness_score": 0.04,
    "Regulatory_complexity_index": 0.02,
}

# Validate and build a weight vector aligned to 'criterion_meta'
crit_list = list(criterion_meta["criterion"])
w_direct = np.array([direct_weights[c] for c in crit_list], dtype=float)
w_sum = w_direct.sum()
if not np.isclose(w_sum, 1.0):
    raise ValueError(f"Direct weights must sum to 1. Current sum = {w_sum:.4f}")
w_direct


In [ ]:

# --- Option B: Entropy weighting (optional) ---
# Based on normalized scores (0-100). Higher dispersion -> larger weight.

def entropy_weights(df_norm: pd.DataFrame, meta: pd.DataFrame) -> np.ndarray:
    X = df_norm[meta["criterion"]].astype(float).values
    # Convert to proportions per criterion
    # Add a small epsilon to avoid log(0)
    eps = 1e-12
    P = (X + eps) / (X.sum(axis=0, keepdims=True) + eps * X.shape[0])
    # Entropy per criterion
    k = 1.0 / np.log(X.shape[0])
    E = -k * (P * np.log(P + eps)).sum(axis=0)
    D = 1 - E                     # degree of diversification
    w = D / D.sum()
    return w

w_entropy = entropy_weights(normalized, criterion_meta)
w_entropy_dict = {c: float(w) for c, w in zip(criterion_meta["criterion"], w_entropy)}
print("Entropy weights (sum to 1):")
w_entropy_dict



> ✅ **Choose your weighting approach:** By default, the lab will use `direct_weights`. To switch to entropy, set `USE_ENTROPY = True` below.


In [ ]:

USE_ENTROPY = False  # ← set to True to use entropy weights
weights = w_entropy if USE_ENTROPY else np.array([direct_weights[c] for c in criterion_meta["criterion"]], dtype=float)
print("Using weights:", {c: float(w) for c, w in zip(criterion_meta['criterion'], weights)})



## 4) Aggregate index

We compute a composite score for each alternative:

- **Weighted Sum (SAW):** `Index = Σ (weight_c × normalized_score_c)` → range 0–100.
- **(Optional) TOPSIS:** distance to the ideal best vs. ideal worst in the normalized‑weighted space.

> 📌 *Interpretation tip:* Weighted sums are easy to explain to policymakers. TOPSIS adds a geometric perspective that can reveal “balanced” options.


In [ ]:

def aggregate_weighted_sum(df_norm: pd.DataFrame, meta: pd.DataFrame, w: np.ndarray) -> pd.DataFrame:
    crit = list(meta["criterion"])
    scores = df_norm[crit].values
    index = (scores * (w * 1.0)).sum(axis=1) / 1.0
    out = df_norm[["Alternative"]].copy()
    out["MCA_WeightedSum_0to100"] = index
    return out

agg_ws = aggregate_weighted_sum(normalized, criterion_meta, weights)
agg_ws.sort_values("MCA_WeightedSum_0to100", ascending=False)


In [ ]:

# Optional: TOPSIS implementation on 0–100 normalized scores
def topsis(df_norm: pd.DataFrame, meta: pd.DataFrame, w: np.ndarray) -> pd.DataFrame:
    crit = list(meta["criterion"])
    X = df_norm[crit].astype(float).values
    # Step 1: Normalize by vector norm per criterion
    norm = np.linalg.norm(X, axis=0)
    Xn = X / np.where(norm == 0, 1, norm)
    # Step 2: Apply weights
    W = w / w.sum()
    V = Xn * W
    # Step 3: Determine ideal best/worst (since scores are already benefit-type 0–100)
    ideal_best = V.max(axis=0)
    ideal_worst = V.min(axis=0)
    # Step 4: Distances
    d_pos = np.sqrt(((V - ideal_best)**2).sum(axis=1))
    d_neg = np.sqrt(((V - ideal_worst)**2).sum(axis=1))
    # Step 5: Closeness
    c_star = d_neg / (d_pos + d_neg + 1e-12)
    out = df_norm[["Alternative"]].copy()
    out["TOPSIS_Closeness_0to1"] = c_star
    return out

agg_topsis = topsis(normalized, criterion_meta, weights)
agg_topsis.sort_values("TOPSIS_Closeness_0to1", ascending=False)



## 5) Results and conclusions

This section compiles results, shows rankings, and asks you to reflect on **limitations**, **trade‑offs**, and **policy implications**.

> 🧪 *Sensitivity tip:* Try different weight sets (stakeholder views) and compare rankings. If rankings flip easily, your recommendation should emphasize **uncertainty** and **robustness**.


In [ ]:

# Merge results
res = normalized.merge(agg_ws, on="Alternative").merge(agg_topsis, on="Alternative")

# Rank (higher is better)
res["Rank_WS"] = res["MCA_WeightedSum_0to100"].rank(ascending=False, method="min").astype(int)
res["Rank_TOPSIS"] = res["TOPSIS_Closeness_0to1"].rank(ascending=False, method="min").astype(int)

# Sort by weighted sum
res_sorted = res.sort_values(["Rank_WS", "Rank_TOPSIS"])
res_sorted.reset_index(drop=True, inplace=True)
res_sorted


In [ ]:

# Plot: Weighted Sum scores
plt.figure()
plt.bar(res_sorted["Alternative"], res_sorted["MCA_WeightedSum_0to100"])
plt.title("MCA Weighted Sum (0–100)")
plt.xlabel("Alternative")
plt.ylabel("Score")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()


In [ ]:

# Plot: TOPSIS closeness
plt.figure()
plt.bar(res_sorted["Alternative"], res_sorted["TOPSIS_Closeness_0to1"])
plt.title("TOPSIS Closeness (0–1)")
plt.xlabel("Alternative")
plt.ylabel("Closeness")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()



### (Optional) One‑way sensitivity on a selected weight

Vary one criterion's weight and track the ranking stability. This is helpful for stakeholder discussions (e.g., “What if we prioritise jobs more?”).

In [ ]:

def one_way_weight_sensitivity(df_norm, meta, base_weights, vary_criterion, sweep=np.linspace(0.0, 0.5, 11)):
    crit = list(meta["criterion"])
    base = base_weights.copy().astype(float)
    idx = crit.index(vary_criterion)
    alts = df_norm["Alternative"].tolist()
    history = {a: [] for a in alts}
    for w_var in sweep:
        w = base.copy()
        # re-distribute remaining weight proportionally across other criteria
        remainder = 1.0 - w_var
        others = np.delete(np.arange(len(crit)), idx)
        base_others = base[others]
        if base_others.sum() == 0:
            w[others] = remainder / len(others)
        else:
            w[others] = remainder * (base_others / base_others.sum())
        w[idx] = w_var
        ws = aggregate_weighted_sum(df_norm, meta, w)
        ordering = ws.sort_values("MCA_WeightedSum_0to100", ascending=False)["Alternative"].tolist()
        for a in alts:
            history[a].append(ordering.index(a) + 1)  # rank (1 = best)
    return sweep, history

# Example: vary weight of GHG_2050_MtCO2e
crit_to_vary = "GHG_2050_MtCO2e"
sweep, hist = one_way_weight_sensitivity(normalized, criterion_meta, weights, crit_to_vary)

plt.figure()
for a, ranks in hist.items():
    plt.plot(sweep, ranks, marker="o", label=a)
plt.gca().invert_yaxis()  # rank 1 at top
plt.xlabel(f"Weight assigned to {crit_to_vary}")
plt.ylabel("Rank (1 = best)")
plt.title("One‑way Sensitivity of Ranking")
plt.legend()
plt.tight_layout()
plt.show()



## 6) Reporting prompts (fill in)

Use these prompts to document your reasoning and communicate to policymakers.

**6.1 Decision framing**
- Country and sectoral scope:  
- Alternatives compared (with 1–2 line description each):  
- Primary policy question (what trade‑off are we informing?):  

**6.2 Criteria justification**
- List each criterion, unit, data source, and why it matters (link to NDCs, EU targets, SDGs).  
- Note any overlaps/correlations and why both are kept or how you de‑duplicated.

**6.3 Normalization choices**
- Why min–max? If you used external anchors (e.g., EU benchmark), explain them.  
- Any transformations (log, capping) and why.

**6.4 Weights**
- Whose priorities do the weights represent (government, stakeholders, public survey)?  
- Provide at least one alternative weight set and discuss robustness.

**6.5 Results & interpretation**
- Key ranking insights (consistency between Weighted Sum and TOPSIS?).  
- Where do trade‑offs appear (e.g., lower bills vs. higher GHG)?  
- What surprised you?

**6.6 Limitations**
- Data gaps, model uncertainty (e.g., ECEMF model assumptions), transferability limits.  
- Institutional feasibility and social acceptance uncertainties.

**6.7 Policy recommendations**
- Near‑term actions (next 3–5 years) and long‑term milestones to 2050.  
- Risk management: what if your preferred option underperforms on a critical criterion?  
- Synergies/trade‑offs with broader sustainability goals and international benchmarks.

> 🧩 *Reminder:* The SDGs target 2030, but your indicators should track progress along the path to **2050**. Document how interim milestones translate into your criteria.



### (Optional) Pairwise comparison helper (AHP‑style)

If you prefer pairwise judgments, use the helper below to derive weights. **Use with care**; keep the number of criteria modest and check consistency.


In [ ]:

import numpy as np
import pandas as pd

def ahp_weights(pairwise_matrix: np.ndarray) -> np.ndarray:
    # Power method on positive reciprocal matrix
    vals, vecs = np.linalg.eig(pairwise_matrix)
    max_idx = np.argmax(vals.real)
    w = vecs[:, max_idx].real
    w = np.abs(w)
    w = w / w.sum()
    return w

# Example skeleton (fill with your judgments; matrix must be reciprocal and positive):
# crit = list(criterion_meta["criterion"])
# n = len(crit)
# P = np.ones((n, n))
# # Fill upper triangle, e.g., P[i, j] = how many times i is preferred over j (1..9)
# # Then set P[j, i] = 1/P[i, j]
# w_ahp = ahp_weights(P)
# print({c: float(w) for c, w in zip(crit, w_ahp)})



## Save a snapshot of your current results

Use this cell to export the key tables to Excel for inclusion in your report.


In [ ]:

with pd.ExcelWriter("MCA_outputs.xlsx", engine="openpyxl") as xl:
    raw_data.to_excel(xl, sheet_name="raw_data", index=False)
    criterion_meta.to_excel(xl, sheet_name="criterion_meta", index=False)
    normalized.to_excel(xl, sheet_name="normalized_0to100", index=False)
    res_sorted.to_excel(xl, sheet_name="results", index=False)

print("Saved: MCA_outputs.xlsx")
try:
    from google.colab import files
    files.download("MCA_outputs.xlsx")
except Exception as e:
    print("If download doesn't start automatically, the file is in the working directory.")



---

### Tips for excellent submissions

- **Traceability:** Every number should have a source or rationale.
- **Plural perspectives:** Reflect at least two weighting schemes (e.g., government vs. civil society).
- **Robustness:** Use sensitivity analysis to test how rankings change.
- **Clarity:** Prefer simple visuals and plain language for policy audiences.
- **Ethics & equity:** Consider distributional impacts (e.g., energy poverty, regional jobs).

Good luck — and have fun exploring the trade‑space! ✨
